In [1]:
import pandas as pd
from openpyxl.styles import PatternFill, NamedStyle, Alignment, Font, Border, Side
from openpyxl.utils.cell import get_column_letter

In [2]:
dummy_data = [
    {"Date": "2023-01-01", "Region": "North", "Product": "A", "Units Sold": 1000, "Revenue": 10000},
    {"Date": "2023-01-01", "Region": "South", "Product": "B", "Units Sold": 1500, "Revenue": 25000},
    {"Date": "2023-01-02", "Region": "East", "Product": "C", "Units Sold": 800, "Revenue": 12000},
    {"Date": "2023-01-02", "Region": "West", "Product": "A", "Units Sold": 1200, "Revenue": 15000},
    {"Date": "2023-01-03", "Region": "North", "Product": "B", "Units Sold": 900, "Revenue": 18000},
    {"Date": "2023-01-03", "Region": "South", "Product": "C", "Units Sold": 1100, "Revenue": 13000},
    {"Date": "2023-01-04", "Region": "East", "Product": "A", "Units Sold": 1300, "Revenue": 20000},
    {"Date": "2023-01-04", "Region": "West", "Product": "B", "Units Sold": 1000, "Revenue": 15000},
    {"Date": "2023-01-05", "Region": "North", "Product": "C", "Units Sold": 700, "Revenue": 9000},
    {"Date": "2023-01-05", "Region": "South", "Product": "A", "Units Sold": 1800, "Revenue": 28000}
]
# Calculate Variance
for i in range(1, len(dummy_data)):
    current_revenue = dummy_data[i]["Revenue"]
    previous_revenue = dummy_data[i - 1]["Revenue"]
    variance = current_revenue - previous_revenue
    dummy_data[i]["Variance"] = variance
# Create the dataframe
df = pd.DataFrame.from_dict(dummy_data)
# Remove null values
df.fillna(0, inplace=True)
df

,Date,Region,Product,Units Sold,Revenue,Variance
0,2023-01-01,North,A,1000,10000,0.0
1,2023-01-01,South,B,1500,25000,15000.0
2,2023-01-02,East,C,800,12000,-13000.0
3,2023-01-02,West,A,1200,15000,3000.0
4,2023-01-03,North,B,900,18000,3000.0
5,2023-01-03,South,C,1100,13000,-5000.0
6,2023-01-04,East,A,1300,20000,7000.0
7,2023-01-04,West,B,1000,15000,-5000.0
8,2023-01-05,North,C,700,9000,-6000.0
9,2023-01-05,South,A,1800,28000,19000.0


In [39]:
with pd.ExcelWriter("Style Excel.xlsx", engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="Sample1", index=False, freeze_panes=(2,0))
    
    # Workbook
    workbook = writer.book
    # Worksheet
    worksheet = workbook["Sample1"]

    # Insert a column and a row 
    # Openpyxl is not zero indexed.
    worksheet.insert_cols(1)
    worksheet.insert_rows(1)

    # Minimize the size of row 1:
    worksheet.row_dimensions[1].height= 21.0
    # worksheet.row_dimensions[1].hidden = True # The row can also be hidden

    # AutoFit column width
    for column in worksheet.columns:
        max_length = max(len(str(cell.value)) for cell in column)
        adjusted_width = (max_length + 2) * 1.0
        worksheet.column_dimensions[column[0].column_letter].width = adjusted_width
    
    # Adjust the size of column "A"
    worksheet.column_dimensions["A"].width = 4.0
    
    add_comma = NamedStyle(name="num_comma", number_format="#,##0")
   
    for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=2, max_col=worksheet.max_column):
        for cell in row:
            # Add the , to separate the numbers and make them readable
            cell.style = add_comma
            cell.alignment = Alignment(horizontal="center")
            
        # Add all borders to all the cells
        # Import Border and Side from openpyxl.styles
        cell.border = Border(left=Side(border_style="thin"), 
                             right=Side(border_style="thin"),
                             top=Side(border_style="thin"),
                             bottom=Side(border_style="thin"))
        
    # Iterate through rows and cells to apply styling
    for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=2, max_col=worksheet.max_column):
        for cell in row:        
            # Add borders to all cells for a neat appearance
            cell.border = Border(left=Side(border_style="thin"), 
                                right=Side(border_style="thin"),
                                top=Side(border_style="thin"),
                                bottom=Side(border_style="thin"))
    # Add a thick outer border
    # 1. Add a thick outer border to the leftmost column
    for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=2, max_col=2):
        for cell in row:
            cell.border = Border(left=Side(border_style="medium"), 
                                  right=Side(border_style="thin"),
                                  top=Side(border_style="thin"),
                                  bottom=Side(border_style="thin"))

    # 2. Add a thick outer border to the rightmost column
    for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=worksheet.max_column, max_col=worksheet.max_column):
        for cell in row:
            cell.border = Border(left=Side(border_style="thin"), 
                                  right=Side(border_style="medium"),
                                  top=Side(border_style="thin"),
                                  bottom=Side(border_style="thin"))

    # 3. Add a thick outer border to the top row
    for row in worksheet.iter_rows(min_row=2, max_row=2, min_col=2, max_col=worksheet.max_column):
        for cell in row:
            cell.border = Border(left=Side(border_style="thin"), 
                                  right=Side(border_style="thin"),
                                  top=Side(border_style="medium"),
                                  bottom=Side(border_style="thin"))

    # Add a thick outer border to the bottom row
    for row in worksheet.iter_rows(min_row=worksheet.max_row, max_row=worksheet.max_row, min_col=2, max_col=worksheet.max_column):
        for cell in row:
            cell.border = Border(left=Side(border_style="thin"), 
                          right=Side(border_style="thin"),
                          top=Side(border_style="thin"),
                          bottom=Side(border_style="medium"))

    # 4. Add a thick border to the top-left cell that was overwritten
    for row in worksheet.iter_rows(min_row=2, max_row=2, min_col=2, max_col=2):
        for cell in row:
            cell.border = Border(left=Side(border_style="medium"), 
                                  top=Side(border_style="medium"))

    # Add a thick border to the bottom-left cell that was overwritten
    for row in worksheet.iter_rows(min_row=worksheet.max_row, max_row=worksheet.max_row, min_col=2, max_col=2):
        for cell in row:
            cell.border = Border(left=Side(border_style="medium"), 
                                  bottom=Side(border_style="medium"))

    # 5. Add a thick border to the top-right cell that was overwritten
    for row in worksheet.iter_rows(min_row=2, max_row=2, min_col=worksheet.max_column, max_col=worksheet.max_column):
        for cell in row:
            cell.border = Border(right=Side(border_style="medium"), 
                                  top=Side(border_style="medium"))

    # 6. Add a thick border to the bottom-right cell that was overwritten
    for row in worksheet.iter_rows(min_row=worksheet.max_row, max_row=worksheet.max_row, min_col=worksheet.max_column, max_col=worksheet.max_column):
        for cell in row:
            cell.border = Border(right=Side(border_style="medium"), 
                                  bottom=Side(border_style="medium"))
                    
    # Add font color to the variance column and make the change the size of the values and  the boldness
    # Red for values less than 0 and green for values greater than 0
    # In this for loop we are targeting column 7 only.
    # NOTE:1. Openpyxl is not zero indexed
    # 2. To change the font color, size, boldness etc, you need to import Font from openpyxl.styles
    # 3. Colors must be aRGB hex values
    
    for row in worksheet.iter_rows(min_row=3, max_row=worksheet.max_row, min_col=worksheet.max_column, max_col=worksheet.max_column):
      for cell in row:
        if cell.value < 0:
          cell.font = Font(color="FF0000", size=12, bold=True)
        else:
          cell.font = Font(color="00B050", size=12, bold=True)
            
    # Change the color fill of the header row
    for row in worksheet.iter_rows(min_row=2, max_row=2, min_col=2, max_col=worksheet.max_column):
      for cell in row:
        cell.fill = PatternFill(fill_type="solid", start_color="CC99FF", end_color="CC99FF")
        
    # Make the fill color for column `Variance` different
    for row in worksheet.iter_rows(min_row=2, max_row=2, min_col=2, max_col=worksheet.max_column):
        for cell in row:
            if cell.value == "Variance":
                cell.fill = PatternFill(fill_type="solid", start_color="FFFF00", end_color="FFFF00")
                
    # Distinguish rows where `Region` is `East`
    for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=3, max_col=3):
        for cell in row:
            if cell.value == "East":
                for row in worksheet.iter_rows(min_row=row[0].row, max_row=row[0].row, min_col=2, max_col=worksheet.max_column):
                    for cell in row:
                        cell.fill = PatternFill(fill_type="solid", start_color="DAEEF3", end_color="DAEEF3")
                                
    # Add a tab color
    worksheet.sheet_properties.tabColor = "002060"
      
      # Adding a value to a cell
    worksheet.cell(1,1).value = "STYLING EXCEL SHEETS"
    worksheet.cell(1,1).font = Font(bold=True, color="FF0000", size=13)
    worksheet.cell(1,1).alignment = Alignment(horizontal="left")


In [ ]:
# If you had merged cells, you can unmerge them using this code

# Unmerge any merged cells
for cells in list(worksheet.merged_cells):
    worksheet.unmerge_cells(range_string=str(cells))

# If you wish to merge cells you can use the code below
 worksheet.merge_cells(start_row=2, end_row=2, start_column=3, end_column=5)
 

In [ ]:
# Remove the applied styles in your sheet
for rows in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=2, max_col=worksheet.max_column):
    for cell in rows:
        cell.style = "Normal"  


In [ ]:
# Move the range one row down
last_column_letter = get_column_letter(worksheet.max_column)
worksheet.move_range(f'B2:{last_column_letter}2', rows=1)


- The decision to add a comma before applying font styles in our variance column wasn't random- it was a strategic move. This careful consideration stems from the fact that if you add a comma after applying the font, the comma style can potentially overwrite the font settings.
- In scenarios where one style might unintentionally overwrite another, preserving the previous style becomes crucial. This becomes even more relevant if you had previously styled your Data Frame using .styler and wish to retain those existing styles.

In [ ]:
# This color is stored before applying the `num_comma_style` because it will be overwritten by it
existing_font_color = cell.font.color
existing_font_weight = cell.font.bold
existing_alignment = cell.alignment.horizontal 

# Add the comma to separate the numbers and enhance readability
cell.style = num_comma_style

# Reapply the font color and bold style
cell.font = Font(color=existing_font_color, bold=existing_font_weight)